In [1]:
%pylab notebook --no-import-all
%matplotlib notebook
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [3]:
# class Voter:
#     def __init__(self, num_parties):
#         self.schema = np.zeros(num_parties)
        
#     def vote()

In [37]:
import operator
from numba import jit

candidates = {'A':0.34593, 'B':0.38819, 'C':0.14528, 'D':0.05823, 'E':0.06237}

@jit
def sort_dict(a_dict):
    return sorted(a_dict.items(), key=operator.itemgetter(1), reverse=True)

def check_probabilites(d):
    # the dictionary's identity should have already been checked
    s = 0
    for k, v in d:
        s += v
    
    return s <= 1

@jit
def rezip_list(l):
    d = {}
    
    for i in range(len(l)):
        d[l[i][0]] = l[i][1]
    
    return d

def pop_and_scale(d, key):
    norm = 1 - d[key]
    
    del d[key]
    
    for (k,v) in d.items():
        d[k] = v / norm
    
    return d

@jit
def vote_iterate(d, n):
    if (type(d) is not dict):
        raise Exception('Selected candidates is not a dictionary')
        
    if len(d) < n:
        raise Exception('Not enough candidates for selection amount')
    
    ordered = sort_dict(d)
    
    if (not check_probabilites(ordered)):
        raise Exception('Probabilities overflow normalization condition')
    
    i   = 0
    out = []
    
    while i < n:
        # first, pick a candidate based on uniform random dist compared to candidate probabilty weights
        vote = np.random.random_sample(1)
        
        s = 0
        selected = 'NOBODY'
        
        # find range where vote landed
        for k, v in ordered:
            s += v
            
            if s > vote:
                selected = k
                break
                
        # vote selected; reduce array and continue selection
        out.append(selected)
        ordered = sort_dict(pop_and_scale(rezip_list(ordered), selected))
        i += 1
    
    return out
        
def build_sample(roster, n):
    # run a sample based on n iterations of voting; TODO: include multiple voter types with different weights
    # the roster is a list of all candidates in string form
    
    # init the vote tally
    d = {}
    
    for i in range(len(roster)):
        d[roster[i]] = 0
    
    for i in range(n):
        votes = vote_iterate(candidates, 2) # magic numbers atm
        
        for j in range(len(votes)):
            d[votes[j]] += 1
    
    return d

In [39]:
schema = ['A', 'B','C','D','E']

build_sample(schema, 300000)

{'A': 200901, 'B': 212645, 'C': 100049, 'D': 41784, 'E': 44621}